### Package Dependencies

In [1]:
import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
import seaborn as sns

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### Connect to MongoDB

In [2]:
conn = 'mongodb://localhost:27018'
client = pymongo.MongoClient(conn)

### Pull Data

In [3]:
db = client.eliteprospects

metadata = db.meta_data
player_stats = db.player_data
award_stats = db.awards_data

meta = pd.DataFrame(list(metadata.find()))
players = pd.DataFrame(list(player_stats.find()))
awards = pd.DataFrame(list(award_stats.find()))

### Write/Read CSV

In [4]:
# meta.to_csv('meta.csv')
# players.to_csv('players.csv')
# inter.to_csv('international.csv')
# playoffs.to_csv('playoffs.csv')

# meta = pd.read_csv('~/Analytics/ScoutingWithAI/Misc/meta.csv')
# players = pd.read_csv('~/Analytics/ScoutingWithAI/Misc/players.csv')
# inter = pd.read_csv('~/Analytics/ScoutingWithAI/Notebooks/international.csv')

## Wrangling

### Get the age of each player for each season

In [4]:
# ep_id as an integer
players['ep_id'] = players['ep_id'].astype(int)
meta['ep_id'] = meta['ep_id'].astype(int)
awards['ep_id'] = awards['ep_id'].astype(int)

# In case of dubplicates, drop them
meta = meta.drop_duplicates()
players = players.drop_duplicates()
awards = awards.drop_duplicates()

In [5]:
# Set aside each players birthday
birthdays = meta[['ep_id', 'date_of_birth']]

# Merge the birthdays with each player row in the players and awards tables
players = pd.merge(players, birthdays, on = 'ep_id', how = 'left')
awards = pd.merge(awards, birthdays, on = 'ep_id', how = 'left')

In [7]:
awards.sample(10)

,_id,ep_id,season,award,award_count,date_of_birth
21341,5e1522cd31ffbecc369dacf4,4295,2009-2010,AHL Player of the Week,1,"Mar 17, 1989"
3008,5e150ebe31ffbecc369d57fa,9512,2000-2001,CHL All-Rookie Team,1,"Oct 07, 1983"
40017,5e155af131ffbecc369e0fd8,9012,2014-2015,NHL Playoffs Most Assists by Defenseman,1,"Jul 16, 1983"
33228,5e1555da31ffbecc369df19a,9503,2010-2011,AHL Second All-Star Team,1,"Feb 09, 1985"
19103,5e151ff331ffbecc369da25a,31079,2010-2011,WHL Playoffs Most Assists,1,"Jan 21, 1991"
29079,5e153f7b31ffbecc369dd8d8,322178,2017-2018,U17 WHC Silver Medal,1,"Jan 26, 2001"
16124,5e151bd931ffbecc369d93dc,11829,2011-2012,Slovakia2 Champion,1,"Apr 20, 1989"
36142,5e15573931ffbecc369dfe40,8526,2013-2014,Olympic Gold Medal,1,"May 10, 1985"
12688,5e15169f31ffbecc369d835c,8906,2016-2017,DEL Champion,1,"Jan 17, 1983"
40368,5e155b7031ffbecc369e1184,11536,2004-2005,WHL Playoffs Most Assists,1,"Mar 20, 1985"


In [8]:
players.sample(10)

,_id,ep_id,season,team,league,regular_gp,regular_g,regular_a,regular_pim,regular_pm,team_type,playoffs_gp,playoffs_g,playoffs_a,playoffs_pim,playoffs_pm,date_of_birth
9913,5e155b5031ffbecc369e10ec,11593,2009-10,Milwaukee Admirals,AHL,76,8,10,18,53,club,Playoffs,5,0,0,0,"Nov 16, 1986"
7558,5e15473531ffbecc369dde33,299028,2015-16,Yale Hockey Academy Prep,CSSHL,22,16,21,37,56,club,Playoffs,3,1,3,8,"Dec 29, 1999"
2067,5e1518bc31ffbecc369d88a5,14844,2000-01,Seattle Thunderbirds,WHL,72,7,17,24,197,club,Playoffs,9,0,2,17,"Feb 01, 1981"
5320,5e152fc031ffbecc369dc7b2,118658,2017-18,Rapid City Rush,ECHL,3,0,1,1,6,club,,,,,,"Apr 20, 1995"
5596,5e15325631ffbecc369dcabc,84261,2012-13,Prince George Cougars,WHL,57,1,5,6,64,club,,,,,,"Nov 21, 1995"
7520,5e1546b831ffbecc369dddec,289459,2014-15,Regina Capitals Midget AA,SSMHL,38,5,18,23,34,club,Playoffs,10,1,5,0,"Sep 12, 1999"
1481,5e1514fd31ffbecc369d77a9,19131,1997-98,Swift Current Broncos,WHL,70,9,11,20,50,club,Playoffs,12,1,4,15,"May 16, 1981"
9980,5e155bb031ffbecc369e11ac,17928,2008-09,Oklahoma City Blazers,CHL,30,2,7,9,17,international,,,,,,"May 02, 1983"
2105,5e1518fc31ffbecc369d89ff,12048,2001-02,Brandon Wheat Kings,WHL,71,18,30,48,96,club,Playoffs,19,4,7,20,"Jul 25, 1984"
1470,5e1514de31ffbecc369d779d,15571,2000-01,Kelowna Rockets,WHL,46,6,8,14,21,club,Playoffs,3,1,1,2,"Oct 29, 1983"


In [9]:
# Extract the end of season date for each yyyy-yy season variable
players['end_of_season'] = '20' + players['season'].str.extract(r'-(\d{2})') + '-04-05'
awards['end_of_season'] = '20' + awards['season'].str.extract(r'-\d{2}(\d{2})') + '-04-05'

In [11]:
awards.sample(10)

,_id,ep_id,season,award,award_count,date_of_birth,end_of_season
17057,5e151cda31ffbecc369d9858,13931,2008-2009,U20 WJC (D1|B) Gold Medal,1,"Dec 01, 1990",2009-04-05
3035,5e150efc31ffbecc369d5838,13558,2001-2002,WHL Best Plus/Minus,1,"Dec 17, 1982",2002-04-05
38100,5e1559f131ffbecc369e079a,11572,2011-2012,DEL Champion,1,"Jan 20, 1983",2012-04-05
16644,5e151cd931ffbecc369d96a1,13931,2008-2009,U20 WJC (D1|B) Most Goals,1,"Dec 01, 1990",2009-04-05
12973,5e15173e31ffbecc369d84ce,8902,2002-2003,U20 WJC Silver Medal,1,"Mar 30, 1983",2003-04-05
16155,5e151bd931ffbecc369d9400,11829,2012-2013,Slovakia2 Champion,1,"Apr 20, 1989",2013-04-05
16417,5e151c3a31ffbecc369d9554,24083,2014-2015,ECHL Player of the Week,1,"Mar 31, 1990",2015-04-05
10762,5e15157e31ffbecc369d7afe,10880,2002-2003,CHL Second All-Star Team,1,"Dec 17, 1983",2003-04-05
14132,5e15191c31ffbecc369d8a77,14450,2000-2001,CHL Top Prospects Game,1,"Dec 30, 1982",2001-04-05
37102,5e1558f331ffbecc369e0309,10983,2001-2002,CHL Top Prospects Game,1,"Sep 27, 1983",2002-04-05


In [12]:
# Convert date_of_birth and end_of_season to pandas datetime
players['date_of_birth'] = pd.to_datetime(players['date_of_birth'])
players['end_of_season'] = pd.to_datetime(players['end_of_season'])

awards['date_of_birth'] = pd.to_datetime(awards['date_of_birth'])
awards['end_of_season'] = pd.to_datetime(awards['end_of_season'])

# Subtract end_of_season from date_of_birth to get the age of each player at the end of each season
players['age'] = players['end_of_season'] - players['date_of_birth']
awards['age'] = awards['end_of_season'] - awards['date_of_birth']

# Convert the pandas timedelta object to integer years
players['age'] = round(players['age'].dt.days / 365).astype(int)
awards['age'] = round(awards['age'].dt.days / 365).astype(int)

# Set aside current date
players['current_date'] = pd.to_datetime('today')
awards['current_date'] = pd.to_datetime('today')

# Subtract players birth date from the current date to get the current age
players['current_age'] = players['current_date'] - players['date_of_birth']
awards['current_age'] = awards['current_date'] - awards['date_of_birth']

# Convert the pandas timedelta to integer years
players['current_age'] = round(players['current_age'].dt.days / 365).astype(int)
awards['current_age'] = round(awards['current_age'].dt.days / 365).astype(int)

In [13]:
awards.sample(10)

,_id,ep_id,season,award,award_count,date_of_birth,end_of_season,age,current_date,current_age
25154,5e152a3531ffbecc369dc008,44144,2018-2019,World Championship Top 3 Player on Team,1,1995-01-22,2019-04-05,24,2020-01-07 23:39:17.196910,25
22951,5e15262231ffbecc369db521,95044,2017-2018,World Championship Top 3 Player on Team,1,1995-03-21,2018-04-05,23,2020-01-07 23:39:17.196910,25
31893,5e15559a31ffbecc369dec25,9145,2002-2003,WHL Most Points by Rookie,1,1983-12-08,2003-04-05,19,2020-01-07 23:39:17.196910,36
4456,5e15109b31ffbecc369d5eff,12043,2012-2013,Denmark Player of the Month,1,1983-03-22,2013-04-05,30,2020-01-07 23:39:17.196910,37
27794,5e15364231ffbecc369dd000,43331,2011-2012,U20 WJC (D1A) Most Goals,1,1993-04-02,2012-04-05,19,2020-01-07 23:39:17.196910,27
11624,5e1515ff31ffbecc369d7ec5,11501,2006-2007,ECHL Most Assists,1,1984-08-05,2007-04-05,23,2020-01-07 23:39:17.196910,35
34094,5e15561a31ffbecc369df531,12833,2013-2014,DEL Most Points,1,1984-05-23,2014-04-05,30,2020-01-07 23:39:17.196910,36
10193,5e15153d31ffbecc369d788d,9040,2004-2005,AHL All-Star Game,1,1982-12-13,2005-04-05,22,2020-01-07 23:39:17.196910,37
23011,5e15262331ffbecc369db569,95044,2017-2018,World Championship Top 3 Player on Team,1,1995-03-21,2018-04-05,23,2020-01-07 23:39:17.196910,25
32370,5e1555da31ffbecc369dee2a,9503,2017-2018,KHL Most Points by Foreigner,1,1985-02-09,2018-04-05,33,2020-01-07 23:39:17.196910,35


In [14]:
# Subset the players table to only include players who are over 25 at the time of running
players = players[players['current_age'] >= 25]
awards = awards[awards['current_age'] >= 25]

In [15]:
awards.sample(10)

,_id,ep_id,season,award,award_count,date_of_birth,end_of_season,age,current_date,current_age
35908,5e15573931ffbecc369dfd4c,8526,2004-2005,U20 WJC Best Plus/Minus,1,1985-05-10,2005-04-05,20,2020-01-07 23:39:17.196910,35
11247,5e15159f31ffbecc369d7d14,8663,2011-2012,KHL Gagarin Cup Champion,1,1982-05-07,2012-04-05,30,2020-01-07 23:39:17.196910,38
26983,5e1532b331ffbecc369dcb1a,40235,2008-2009,MMHL Champion,1,1992-09-28,2009-04-05,17,2020-01-07 23:39:17.196910,27
5725,5e15123c31ffbecc369d6506,9067,2000-2001,WHL Champion,1,1982-11-23,2001-04-05,18,2020-01-07 23:39:17.196910,37
18780,5e151f5431ffbecc369da0b3,12565,2007-2008,U18 WJC (D1|A) Most Assists,1,1990-11-19,2008-04-05,17,2020-01-07 23:39:17.196910,29
496,5e150cfe31ffbecc369d4cbe,6042,1996-1997,AMBHL Rookie of the Year,1,1982-01-07,2097-04-05,115,2020-01-07 23:39:17.196910,38
7034,5e15133d31ffbecc369d6adf,11029,2018-2019,ECHL Most Valuable Player,1,1982-09-28,2019-04-05,37,2020-01-07 23:39:17.196910,37
7918,5e15135e31ffbecc369d6e83,9609,2011-2012,KHL Gagarin Cup Clinching Goal,1,1984-06-05,2012-04-05,28,2020-01-07 23:39:17.196910,36
33931,5e1555fa31ffbecc369df47c,14701,2005-2006,ECHL Player of the Week,1,1983-08-20,2006-04-05,23,2020-01-07 23:39:17.196910,36
3275,5e150f5c31ffbecc369d595e,12476,2018-2019,EIHL Cup Champion,1,1983-03-06,2019-04-05,36,2020-01-07 23:39:17.196910,37


In [17]:
awards_grouped = awards.groupby(['ep_id', 'age']).agg({'award_count': 'sum'}).reset_index()
awards_grouped.sample(20)

,ep_id,age,award_count
498,15331,26,15
466,14404,24,18
83,8895,20,96
448,13931,16,80
388,11829,17,27
173,9458,30,120
639,32779,28,25
675,37201,27,17
774,59563,17,15
351,11660,25,44


In [25]:
players = pd.merge(players, awards_grouped, on = ['ep_id', 'age'], how = 'left')


In [26]:
players.sample(10)

,_id,ep_id,season,team,league,regular_gp,regular_g,regular_a,regular_pim,regular_pm,team_type,playoffs_gp,playoffs_g,playoffs_a,playoffs_pim,playoffs_pm,date_of_birth,end_of_season,age,current_date,current_age,award_count
4755,5e152ee431ffbecc369dc70b,83623,2011-12,Vancouver Giants,WHL,10,1,0,1,2,club,,,,,,1995-04-07,2012-04-05,17,2020-01-07 23:39:17.190917,25,NaN
1695,5e15161f31ffbecc369d7f28,22615,2008-09,Hijs Hokij Den Haag,Netherlands,26,10,12,22,57,club,Playoffs,10,1,3,10,1982-04-30,2009-04-05,27,2020-01-07 23:39:17.190917,38,13.0
3434,5e15215031ffbecc369da551,37227,2009-10,Red Deer Rebels,WHL,72,4,28,32,91,club,Playoffs,4,0,1,0,1989-06-28,2010-04-05,21,2020-01-07 23:39:17.190917,31,NaN
2928,5e151e1731ffbecc369d9def,11658,2004-05,Canada Pacific U17,WHC-17,6,2,4,6,0,international,,,,,,1988-03-14,2005-04-05,17,2020-01-07 23:39:17.190917,32,20.0
6699,5e1557b831ffbecc369e0117,9097,2001-02,Moose Jaw Warriors,WHL,72,8,12,20,56,club,Playoffs,12,0,3,11,1984-05-25,2002-04-05,18,2020-01-07 23:39:17.190917,36,NaN
3471,5e15219031ffbecc369da64b,11103,2009-10,HC Vitkovice,Czech,26,1,2,3,6,international,,,,,,1989-02-08,2010-04-05,21,2020-01-07 23:39:17.190917,31,43.0
401,5e150ebd31ffbecc369d56c9,9512,2005-06,Milwaukee Admirals,AHL,23,17,16,33,44,international,Playoffs,14,6,16,20,1983-10-07,2006-04-05,23,2020-01-07 23:39:17.190917,36,34.0
2858,5e151d9931ffbecc369d9ca6,23797,2011-12,Univ. of Alberta,USports,26,0,12,12,60,club,Playoffs,3,2,2,12,1988-07-29,2012-04-05,24,2020-01-07 23:39:17.190917,31,NaN
4278,5e15287b31ffbecc369dba48,53542,2010-11,Canada Pacific U17,WHC-17,6,0,12,12,2,international,,,,,,1994-07-25,2011-04-05,17,2020-01-07 23:39:17.190917,25,115.0
1362,5e15141e31ffbecc369d7234,83127,2006-07,Bloomington PrairieThunder,UHL,24,4,6,10,28,club,,,,,,1981-01-22,2007-04-05,26,2020-01-07 23:39:17.190917,39,NaN


### Manage +/-

In [ ]:
# Deal with mis-typed values in the plus_minus column
players['plus_minus'] = players['plus_minus'].replace('--6', '-6')

# Impute 0 inplace of - in plus_minus (Fortunately, plus_minus is gaussian and 0 is the mean)
players.replace('-', 0, inplace=True)
players['plus_minus'] = players['plus_minus'].astype(int)

### Reshape quantitative data

In [ ]:
ids = players['ep_id'].unique()

types = {
    'assists': 'int64',
    'games_played': 'int64',
    'goals': 'int64',
    'penalty_min': 'int64',
    'plus_minus': 'int64',
    'age': 'object'
}

new_df = pd.DataFrame()

for id in ids:
        
    df = players[players['ep_id'] == id].drop_duplicates()
    df = df.astype(types)
    
    # Have to groupby each players age to account for in-season trades
    df = df.groupby(['age', 'ep_id']).agg({
        'games_played': 'sum', 
        'goals': 'sum', 
        'assists': 'sum', 
        'penalty_min': 'sum',
        'plus_minus': 'mean'
    }).reset_index()    
    
    # Pivot the data frame to form a single row vector for each player
    df = df.pivot('ep_id', 'age')
    
    # Column Labels    
    # Manage the pandas multiindex to re-label each column for each stat and age
    df.columns = [''.join(str(col)) for col in df.columns]
    
    # Rename column labels for clarity
    labels = []
    
    for col in df:
    
        label = re.findall(r"'(.+)'", col)
        stat = re.findall(r',\s(\d{2})', col)
    
        labels.append(label[0] + '_' + stat[0])
        
    df.columns = labels    
    
    # Append each row of player data to the last
    new_df = pd.concat([new_df, df], axis=0, sort=False)

### Reshape qualitative

In [ ]:
ids = players['ep_id'].unique()

qual_df = pd.DataFrame()

for id in ids:
    
    df = players[players['ep_id'] == id].reset_index().sort_values('age')
    df = df.drop(['index', '_id'], 1).drop_duplicates()
    
    ages = []
    teams = []
    leagues = []
    age_test = []

    high_gp = 0

    for row in df.iterrows():    

        if row[1]['age'] not in age_test:

            age_test.append(row[1]['age'])

            age = row[1]['age']
            team = row[1]['team']
            league = row[1]['league']

            high_gp = int(row[1]['games_played'])

            ages.append(age)
            teams.append(team)
            leagues.append(league)

        elif row[1]['age'] in age_test:

            if int(row[1]['games_played']) > high_gp:

                age = row[1]['age']
                team = row[1]['team']
                league = row[1]['league']

                high_gp = int(row[1]['games_played'])

                ages = ages[:-1]
                teams = teams[:-1]
                leagues = leagues[:-1]

                ages.append(age)
                teams.append(team)
                leagues.append(league)

    test_df = pd.DataFrame(zip(ages, teams, leagues), columns=['age', 'team', 'league'])

    test_df['ep_id'] = row[1]['ep_id']

    test_df = test_df.pivot('ep_id', 'age')

    test_df.columns = [''.join(str(col)) for col in test_df.columns]

    # Rename column labels for clarity
    labels = []

    for col in test_df:

        label = re.findall(r"'(.+)'", col)
        stat = re.findall(r',\s(\d{2})', col)

        labels.append(label[0] + '_' + stat[0])

    test_df.columns = labels
    
    qual_df = pd.concat([qual_df, test_df], axis=0, sort=False)

### Merge qualitative and quantitative dataframes

In [ ]:
combined = pd.merge(new_df, qual_df, on='ep_id', how='left').reset_index()

### Combine Meta and Combined dataframes

In [ ]:
main = pd.merge(meta, combined, on = 'ep_id', how = 'left')
    
main = main.reset_index()
main = round(main.drop(['_id', 'index'], 1))

### Save and inspect data

In [ ]:
#main.to_csv('main.csv')